In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import VearchDb
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoModel, AutoTokenizer

# repalce to your local model path
model_path = ""  
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda(0)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/export/anaconda3/envs/vearch_liama_index/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
query = "你好!"
response, history = model.chat(tokenizer, query, history=[])
print(f"Human: {query}\nChatGLM:{response}\n")
query = "你知道凌波微步吗，你知道都有谁学会了吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")

Human: 你好!
ChatGLM:你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

Human: 你知道凌波微步吗，你知道都有谁学会了吗?
ChatGLM:凌波微步是一种步法，源自金庸先生的武侠小说《天龙八部》。我是一个人工智能助手，我没有感情和学习能力，所以无法了解关于凌波微步的更多信息。



In [4]:
# Add your local knowledge files
file_path = "Your local file path"
loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()
# split text into sentences and embedding the sentences
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
embedding_path = "your model path"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)

No sentence-transformers model found with name /data/zhx/zhx/langchain-ChatGLM_new/text2vec/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [5]:
#first add your document into vearch vectorstore
!pip3 install pyvearch
vearch = VearchDb.from_documents(
    texts,
    embeddings,
    path_or_url="your_vearch_router_url",
    db_name="your_db",
    table_name="your_table",
)

Looking in indexes: http://mirrors.jd.com/pypi/web/simple
['5613292411219285975', '120290588477549397', '-7527337722553895939']


In [6]:
query = "你知道凌波微步吗，你知道都有谁会凌波微步?"
cluster_res = vearch.similarity_search(query, 3)
for idx, tmp in enumerate(cluster_res): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")

# combine your local knowleadge and query 
context = "".join([tmp.page_content for tmp in cluster_res])
new_query = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context} \n 回答用户这个问题:{query}\n\n"
response, history = model.chat(tokenizer, new_query, history=[])
print(f"********ChatGLM:{response}\n")

####################第1段相关文档####################

午饭过后，段誉又练“凌波微步”，走一步，吸一口气，走第二步时将气呼出，六十四卦走完，四肢全无麻痹之感，料想呼吸顺畅，便无害处。第二次再走时连走两步吸一口气，再走两步始行呼出。这“凌波微步”是以动功修习内功，脚步踏遍六十四卦一个周天，内息自然而然地也转了一个周天。因此他每走一遍，内力便有一分进益。

这般练了几天，“凌波微步”已走得颇为纯熟，不须再数呼吸，纵然疾行，气息也已无所窒滞。心意既畅，跨步时渐渐想到《洛神赋》中那些与“凌波微步”有关的句子：“仿佛兮若轻云之蔽月，飘飘兮若流风之回雪”，“竦轻躯以鹤立，若将飞而未翔”，“体迅飞凫，飘忽若神”，“动无常则，若危若安。进止难期，若往若还”。



百度简介

凌波微步是「逍遥派」独门轻功身法，精妙异常。

凌波微步乃是一门极上乘的轻功，所以列于卷轴之末，以易经八八六十四卦为基础，使用者按特定顺序踏着卦象方位行进，从第一步到最后一步正好行走一个大圈。此步法精妙异常，原是要待人练成「北冥神功」，吸人内力，自身内力已【颇为深厚】之后再练。

####################第2段相关文档####################

午饭过后，段誉又练“凌波微步”，走一步，吸一口气，走第二步时将气呼出，六十四卦走完，四肢全无麻痹之感，料想呼吸顺畅，便无害处。第二次再走时连走两步吸一口气，再走两步始行呼出。这“凌波微步”是以动功修习内功，脚步踏遍六十四卦一个周天，内息自然而然地也转了一个周天。因此他每走一遍，内力便有一分进益。

这般练了几天，“凌波微步”已走得颇为纯熟，不须再数呼吸，纵然疾行，气息也已无所窒滞。心意既畅，跨步时渐渐想到《洛神赋》中那些与“凌波微步”有关的句子：“仿佛兮若轻云之蔽月，飘飘兮若流风之回雪”，“竦轻躯以鹤立，若将飞而未翔”，“体迅飞凫，飘忽若神”，“动无常则，若危若安。进止难期，若往若还”。



百度简介

凌波微步是「逍遥派」独门轻功身法，精妙异常。

凌波微步乃是一门极上乘的轻功，所以列于卷轴之末，以易经八八六十四卦为基础，使用者按特定顺序踏着卦象方位行进，从第一步到最后一步正好行走一个大圈。此步法精妙异常，原是要待人练成「北冥神功」，吸人内力，自身内力已【颇为深厚】之后再练。

######

In [7]:
query = "你知道vearch是什么吗?"
response, history = model.chat(tokenizer, query, history=history)
print(f"Human: {query}\nChatGLM:{response}\n")
vearch_info = [
    "Vearch 是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量，可用于基于个人知识库的大模型应用",
    "Vearch 支持OpenAI, Llama, ChatGLM等模型，以及LangChain库",
    "vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装"
]
vearch_source=[
    {'source': '/data/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'},
    {'source': '/data/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'},
    {'source': '/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/tlbb/three_body.txt'},
]
vearch.add_texts(vearch_info,vearch_source)

Human: 你知道vearch是什么吗?
ChatGLM:Vearch是一种向量分解法，用于将矩阵分解为若干个基向量的乘积。具体来说，Vearch方法将矩阵分解为列向量，然后将这些列向量拼接起来得到一个新的列向量，这个新的列向量就是原矩阵的一个基向量。基向量可以根据需要选择，也可以通过某种方式生成。Vearch方法在矩阵分解中具有较高的计算效率，尤其是在处理大型矩阵时。

['2905408703429571146', '-4146045014700199589', '3823416425172812985']


['2905408703429571146', '-4146045014700199589', '3823416425172812985']

In [9]:
query3_c = "你知道vearch是什么吗?"
res1_c = vearch.similarity_search(query3_c, 3)
for idx, tmp in enumerate(res1_c): 
    print(f"{'#'*20}第{idx+1}段相关文档{'#'*20}\n\n{tmp.page_content}\n")
context1_C = "".join([tmp.page_content for tmp in res1_c])
new_query1_c = f"基于以下信息，尽可能准确的来回答用户的问题。背景信息:\n {context1_C} \n 回答用户这个问题:{query3_c}\n\n"
response_c, history_c = model.chat(tokenizer, new_query1_c, history=[])
print(f"***************ChatGLM:{response_c}\n")

####################第1段相关文档####################

Vearch 是一款存储大语言模型数据的向量数据库，用于存储和快速搜索模型embedding后的向量，可用于基于个人知识库的大模型应用

####################第2段相关文档####################

Vearch 支持OpenAI, Llama, ChatGLM等模型，以及LangChain库

####################第3段相关文档####################

vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装

***************ChatGLM:是的，Vearch是一款基于C语言和Go语言开发的向量数据库，旨在存储和快速搜索模型embedding后的向量，并支持OpenAI、ChatGLM等模型。它还提供Python接口，可以直接通过pip安装。Vearch可以用于基于个人知识库的大模型应用，如问答系统、推荐系统等。



In [10]:
# delete and get function need to maintian docids 
res_cluster=vearch.delete(['5613292411219285975', '120290588477549397','-4146045014700199589'])
print("delete vearch cluster docid", res_cluster)
get_id_doc=vearch.get(['-7527337722553895939','3823416425172812985'])
print("get existed docid", get_id_doc)

delete vearch cluster docid True
get existed docid {'-7527337722553895939': Document(page_content='午饭过后，段誉又练“凌波微步”，走一步，吸一口气，走第二步时将气呼出，六十四卦走完，四肢全无麻痹之感，料想呼吸顺畅，便无害处。第二次再走时连走两步吸一口气，再走两步始行呼出。这“凌波微步”是以动功修习内功，脚步踏遍六十四卦一个周天，内息自然而然地也转了一个周天。因此他每走一遍，内力便有一分进益。\n\n这般练了几天，“凌波微步”已走得颇为纯熟，不须再数呼吸，纵然疾行，气息也已无所窒滞。心意既畅，跨步时渐渐想到《洛神赋》中那些与“凌波微步”有关的句子：“仿佛兮若轻云之蔽月，飘飘兮若流风之回雪”，“竦轻躯以鹤立，若将飞而未翔”，“体迅飞凫，飘忽若神”，“动无常则，若危若安。进止难期，若往若还”。\n\n\n\n百度简介\n\n凌波微步是「逍遥派」独门轻功身法，精妙异常。\n\n凌波微步乃是一门极上乘的轻功，所以列于卷轴之末，以易经八八六十四卦为基础，使用者按特定顺序踏着卦象方位行进，从第一步到最后一步正好行走一个大圈。此步法精妙异常，原是要待人练成「北冥神功」，吸人内力，自身内力已【颇为深厚】之后再练。', metadata={'source': '/data/zhx/zhx/langchain-ChatGLM_new/knowledge_base/天龙八部/lingboweibu.txt'}), '3823416425172812985': Document(page_content='vearch 是基于C语言,go语言开发的，并提供python接口，可以直接通过pip安装')}
